In [ ]:
import numpyro
from numpyro.infer import MCMC, NUTS, Predictive
import numpyro.distributions as dist
import numpy as np
import jax.numpy as jnp
from jax import random
from jax import vmap
import matplotlib.pyplot as plt
import pandas as pd
from jax.scipy.stats import norm 
import seaborn as sns

# GOH
Gasser et al 2006(https://doi.org/10.1098/rsif.2005.0073)


Incompressible material assumption
$$\Psi_1(J) = p(1-J)$$
$$\Psi_2(\mathbf{C}) = \frac{\mu}{2}(I_1 -3) +\frac{k_1}{2k_2}[e^{(k_2[\kappa I_1 + (1-3\kappa)I_4-1]^2)}-1]$$
Here $I_4 = \mathbf{C}:\mathbf{M} =\mathbf{C}:  \mathbf{a}_0\otimes \mathbf{a}_0$

Let $\mathbf{H} =\kappa \mathbf{I} + (1-3\kappa)\mathbf{M}$ 

and $H = \mathbf{H}:\mathbf{C}  =\kappa I_1 + (1-3\kappa)I_4$ and $E = [{H}-1]$

$$\mathbf{S}_1  = 2\frac{\partial \Psi_{1}}{\partial \mathbf{C}} = -pC^{-1} $$
$$\mathbf{S}_2  =  2\frac{\partial \Psi_{2}}{\partial \mathbf{C}} = \mu \mathbf{I} + 2k_1 e^{k_2{E}^2}[\kappa \mathbf{I}+(1-3\kappa)\mathbf{M}]$$

Total PK2 stress

$$\mathbf{S} = -p\mathbf{C}^{-1} +  \mu\mathbf{I} + 2k_1 e^{k_2{E}^2}{E}[\kappa \mathbf{I}+(1-3\kappa)\mathbf{M}]$$

Total PK1 stress

$$\mathbf{P} = \mathbf{F}\mathbf{S}$$

In [ ]:
# def neoHook(C1, lmx, lmy, lmz):
#     F = jnp.array([[lmx, 0, 0],
#                     [0, lmy, 0],
#                     [0, 0, lmz]])
#     b = jnp.dot(F,F.T)

#     p = 2*C1*b[2,2]
#     sgm = -p*jnp.eye(3) + 2*C1*b
#     return sgm[0,0], sgm[1,1]
# neoHook_vmap = vmap(neoHook, in_axes=(0,0,0,0), out_axes=(0,0))
# neoHook_vmap2 = vmap(neoHook, in_axes=(None,0,0,0), out_axes=(0,0))

#### Here, I define the function to evaluate the stress considering the GOH model with two groups of fibers

def GOH(par, lmx, lmy, lmz):
    mu, k1, k2, kappa, alpha, k1_2, k2_2, kappa2, alpha2 = par
    mu = 0.05*(jnp.tanh(mu)+1)/2
    k1 = 1000*(jnp.tanh(k1)+1)/2
    k2 = 1000*(jnp.tanh(k2)+1)/2
    kappa = 0.33*(jnp.tanh(kappa)+1)/2
    k1_2 = 1000*(jnp.tanh(k1_2)+1)/2
    k2_2 = 1000*(jnp.tanh(k2_2)+1)/2
    kappa2 = 0.33*(jnp.tanh(kappa2)+1)/2

    mu = 0.05*norm.cdf(mu)
    # k1 = 100*norm.cdf(k1)
    # k2 = 1000*norm.cdf(k2)
    # kappa = 0.33*(jnp.tanh(kappa)+1)/2
    # k1_2 = 100*norm.cdf(k1_2)
    # k2_2 = 1000*norm.cdf(k2_2)
    # kappa2 = 0.33*(jnp.tanh(kappa2)+1)/2

    
    a1 = jnp.array([jnp.cos(alpha),jnp.sin(alpha),0])
    a2 = jnp.array([jnp.cos(alpha2),jnp.sin(alpha2),0])
    M1 = jnp.outer(a1,a1)
    M2 = jnp.outer(a2,a2)

    F = jnp.array([[lmx, 0, 0],
                    [0, lmy, 0],
                    [0, 0, lmz]])
    C = jnp.dot(F.T,F)
    invC = jnp.linalg.inv(C)
    I = jnp.eye(3)
    I1 = C[0,0] + C[1,1] + C[2,2]  ## trace of C
    I4 = jnp.tensordot(C,M1)
    I4_2 = jnp.tensordot(C,M2)

    H1 = kappa*I1+(1-3*kappa)*I4
    H2 = kappa2*I1+(1-3*kappa2)*I4_2
    E1 = H1-1.0
    E2 = H2-1
    S2 = mu*I+2*k1*jnp.exp(k2*E1**2)*E1*(kappa*I+(1-3*kappa)*M1)+2*k1_2*jnp.exp(k2_2*E2**2)*E2*(kappa2*I+(1-3*kappa2)*M2)
    p = S2[2,2]/invC[2,2]
    S = -p*invC+S2

    sgm = jnp.dot(F,jnp.dot(S,F.T))

    return sgm[0,0], sgm[1,1]

GOH_vmap = vmap(GOH, in_axes=(0,0,0,0), out_axes=(0,0))
GOH_vmap2 = vmap(GOH, in_axes=(None,0,0,0), out_axes=(0,0))

In [ ]:
# Load the data
OffX_P10E2A11      = pd.read_csv('TE/PK_lam_OffX_P10E2A1.csv').to_numpy()[:,1:]
OffX_P9E2A11    = pd.read_csv('TE/PK_lam_OffX_P9E2A1.csv').to_numpy()[:,1:]
OffX_3TPA11    = pd.read_csv('TE/PK_lam_OffX_3TPA1.csv').to_numpy()[:,1:]
# OffX_P9E1A11 = pd.read_csv('TE/PK_lam_OffX_P9E1A1.csv').to_numpy()[:,1:]

vecze  = jnp.ones((len(OffX_P10E2A11),1))
vecze2 = jnp.ones((len(OffX_P9E2A11),1))
vecze3 = jnp.ones((len(OffX_3TPA11),1))
# vecze4 = jnp.ones((len(OffX_P9E1A11),1))

OffX_P10E2A1       = jnp.hstack([0*vecze,OffX_P10E2A11])
OffX_P9E2A1     = jnp.hstack([1*vecze2,OffX_P9E2A11])
OffX_3TPA1     = jnp.hstack([2*vecze3,OffX_3TPA11])
# OffX_P9E1A1  = jnp.hstack([3*vecze4,OffX_P9E1A11])

OffXall = jnp.vstack([OffX_P10E2A1,OffX_P9E2A1,OffX_3TPA1
                #    ,OffX_P9E1A1
                  ]).T
                
n_samples=3
data = np.zeros([5,jnp.shape(OffXall)[1]]) # data: [sample id, lambda_x, lambda_y, sigma_xx, sigma_yy] <-- contains this variables as columns

data[0,:]=OffXall[0,:]
data[1,:]=OffXall[1,:]
data[2,:]=OffXall[3,:]
data[3,:]=OffXall[2,:]
data[4,:]=OffXall[4,:]

In [ ]:
for i_sample in [0,1,2]:
    sample_lmx  = data[1,data[0,:]==i_sample]
    sample_lmy  = data[2,data[0,:]==i_sample]
    sample_sgmx = data[3,data[0,:]==i_sample]
    sample_sgmy = data[4,data[0,:]==i_sample]
    plt.plot(sample_lmy, sample_sgmy, '.')

In [ ]:
n_params = 9
def model(i_sample, lmx, lmy, sgmx_gt=None, sgmy_gt=None):
    # Individual parameters
    L_i = numpyro.sample('L_i', dist.Normal(0,1), sample_shape=(n_params,n_params))
    K_i = L_i @ L_i.T
    mu_i = numpyro.sample('mu_i', dist.Normal(0,1), sample_shape=(n_params,))
    with numpyro.plate('plate_i', n_samples):
        params_i = numpyro.sample('params_i', dist.MultivariateNormal(mu_i, K_i))
        
    lmz = 1.0/(lmx*lmy)
    sgmx_pr, sgmy_pr = GOH_vmap(params_i[i_sample], lmx, lmy, lmz)
    sgm_noise = numpyro.sample('sgm_noise', dist.HalfNormal(0.005))
    with numpyro.plate('data', len(lmx)):
        numpyro.sample('obs_sgmx', dist.Normal(sgmx_pr,sgm_noise), obs=sgmx_gt)
        numpyro.sample('obs_sgmy', dist.Normal(sgmy_pr,sgm_noise), obs=sgmy_gt)

In [ ]:
nuts_kernel = NUTS(model)

mcmc = MCMC(nuts_kernel, num_samples=2000, num_warmup=2000) # use 20000 and 200000
rng_key = random.PRNGKey(0)
mcmc.run(rng_key, data[0,:].astype(int), data[1,:], data[2,:], data[3,:], data[4,:])

posterior_samples = mcmc.get_samples()

In [ ]:
fig, ax = plt.subplots(3, 9, figsize=(20, 8))

for i in range(3):
    sns.histplot(np.array(0.05*norm.cdf(posterior_samples["params_i"][:, i, 0])), ax=ax[i,0])
    sns.histplot(np.array(100*norm.cdf(posterior_samples["params_i"][:, i, 1])), ax=ax[i,1])
    sns.histplot(np.array(1000*norm.cdf(posterior_samples["params_i"][:, i, 2])), ax=ax[i,2])
    sns.histplot(np.array(0.33*norm.cdf(posterior_samples["params_i"][:, i, 3])), ax=ax[i,3])
    sns.histplot(np.array(posterior_samples["params_i"][:, i, 4]), ax=ax[i,4])
    sns.histplot(np.array(100*norm.cdf(posterior_samples["params_i"][:, i, 5])), ax=ax[i,5])
    sns.histplot(np.array(1000*norm.cdf(posterior_samples["params_i"][:, i, 6])), ax=ax[i,6])
    sns.histplot(np.array(0.33*norm.cdf(posterior_samples["params_i"][:, i, 7])), ax=ax[i,7])
    sns.histplot(np.array(posterior_samples["params_i"][:, i, 8]), ax=ax[i,8])


# sns.kdeplot(data2, ax=ax[1,1], label='Distribution 2')

ax[0,0].set_title('mu')
ax[0,1].set_title('k1_1')
ax[0,2].set_title('k2_1')
ax[0,3].set_title('kappa1')
ax[0,4].set_title('alpha1')
ax[0,5].set_title('k1_2')
ax[0,6].set_title('k2_2')
ax[0,7].set_title('kappa2')
ax[0,8].set_title('alpha2')

# ax[2,3].spines['left'].set_visible(False)
# ax[0,8].spines['top'].set_visible(False)

# ax[0,0].legend()
# ax[1,0].legend()
# ax[0,1].legend()
# ax[1,1].legend()

# # plt.legend()
# plt.savefig("PosteriorParams_iCTRL.png",dpi=300)
fig.tight_layout(pad=0.1)

plt.show()

In [ ]:
fig, ax = plt.subplots(3, 9, figsize=(20, 8))

for i in range(3):
    sns.kdeplot(np.array(0.05*norm.cdf(posterior_samples["params_i"][:, i, 0])), ax=ax[i,0], bw_method=0.3, label='')
    sns.kdeplot(np.array(100*norm.cdf(posterior_samples["params_i"][:, i, 1])), ax=ax[i,1], bw_method=0.3)
    sns.kdeplot(np.array(1000*norm.cdf(posterior_samples["params_i"][:, i, 2])), ax=ax[i,2], bw_method=0.3)
    sns.kdeplot(np.array(0.33*norm.cdf(posterior_samples["params_i"][:, i, 3])), ax=ax[i,3], bw_method=0.3)
    sns.kdeplot(np.array(posterior_samples["params_i"][:, i, 4]), ax=ax[i,4], bw_method=0.3)
    sns.kdeplot(np.array(100*norm.cdf(posterior_samples["params_i"][:, i, 5])), ax=ax[i,5], bw_method=0.3)
    sns.kdeplot(np.array(1000*norm.cdf(posterior_samples["params_i"][:, i, 6])), ax=ax[i,6], bw_method=0.3)
    sns.kdeplot(np.array(0.33*norm.cdf(posterior_samples["params_i"][:, i, 7])), ax=ax[i,7], bw_method=0.3)
    sns.kdeplot(np.array(posterior_samples["params_i"][:, i, 8]), ax=ax[i,8], bw_method=0.3)

# sns.kdeplot(data2, ax=ax[1,1], label='Distribution 2')

ax[0,0].set_title('mu')
ax[0,1].set_title('k1_1')
ax[0,2].set_title('k2_1')
ax[0,3].set_title('kappa1')
ax[0,4].set_title('alpha1')
ax[0,5].set_title('k1_2')
ax[0,6].set_title('k2_2')
ax[0,7].set_title('kappa2')
ax[0,8].set_title('alpha2')

# ax[2,3].spines['left'].set_visible(False)
# ax[0,8].spines['top'].set_visible(False)

# ax[0,0].legend()
# ax[1,0].legend()
# ax[0,1].legend()
# ax[1,1].legend()

# # plt.legend()
# plt.savefig("PosteriorParams_iCTRL.png",dpi=300)
fig.tight_layout(pad=0.1)

plt.show()

In [ ]:
TableM=np.zeros((3,9))
print(np.shape(TableM))
for i in range(3):
    for j in range(9):
        x=posterior_samples['params_i'][:,i,j]
        if j!=4 and j!=8:
            if j==3 or j==7:
                for m in range(len(x)):
                    x = x.at[m].set(0.33*(jnp.tanh(x[m])+1)/2)
            elif j==1 or j==5:
                for m in range(len(x)):
                    x = x.at[m].set(100*norm.cdf(x[m]))
            elif j==2 or j==6:
                for m in range(len(x)):
                    x = x.at[m].set(1000*norm.cdf(x[m]))
            elif j==0:
                for m in range(len(x)):
                    x = x.at[m].set(0.05*norm.cdf(x[m]))
            else:
                for m in range(len(x)):
                    x = x.at[m].set(jnp.exp(x[m]))

        TableM[i,j]=np.median(x)
        # TableM[i,j]=np.median(x)

print(TableM)


TEcsv = pd.DataFrame(data=TableM, index=None, columns=['mu','k1_1','k2_1','kappa','alpha','k2_1','k2_2','kappa2','alpha2'], dtype=None, copy=False)
TEcsv.to_csv('TEcsv.csv')

In [ ]:
TEcsv = pd.DataFrame(data=TableM, index=None, columns=['mu','k1_1','k2_1','kappa','alpha','k2_1','k2_2','kappa2','alpha2'], dtype=None, copy=False)
TEcsv.to_csv('TEcsv.csv')

In [ ]:
posterior_samples{K}

In [ ]:
predictive = Predictive(model, posterior_samples)
samples_predictive = predictive(random.PRNGKey(1), data[0,:].astype(int), data[1,:], data[2,:], None, None)

In [ ]:
fig, axes = plt.subplots(3,2,figsize=(12,20))


for ax, i_sample in zip(axes, [0,1,2]):
    sgmx_pr = samples_predictive['obs_sgmx']
    sgmy_pr = samples_predictive['obs_sgmy']    

    idx = data[0,:]==i_sample
    x = data[2,idx]
    # print(x)
    sgmx_gt = data[3,idx] ##sigma real
    sgmx_std = sgmx_pr.std(axis=0)[idx] ##sigma predicted
    sgmx_pr = sgmx_pr.mean(axis=0)[idx]
    sgmx_inf = sgmx_pr-sgmx_std
    sgmx_sup = sgmx_pr+sgmx_std

    ax[0].plot(x,sgmx_gt,'.')
    ax[0].plot(x,sgmx_pr)
    ax[0].fill_between(x, sgmx_sup, sgmx_inf, alpha=0.5)
    ax[0].set(xlabel='$\lambda_y$', ylabel='$\sigma_{xx}$')


    sgmy_gt = data[4,idx]
    sgmy_std = sgmy_pr.std(axis=0)[idx]
    sgmy_pr = sgmy_pr.mean(axis=0)[idx]
    sgmy_inf = sgmy_pr-sgmy_std
    sgmy_sup = sgmy_pr+sgmy_std
    ax[1].plot(x,sgmy_gt,'.')
    ax[1].plot(x,sgmy_pr)
    ax[1].fill_between(x, sgmy_sup, sgmy_inf, alpha=0.5)
    ax[1].set(xlabel='$\lambda_y$', ylabel='$\sigma_{yy}$')

fig.suptitle('TE data', fontsize= 20)
plt.savefig("TE_data_i",dpi=300)


In [ ]:
params_i = posterior_samples['params_i']
params_i = params_i[:,:3,:]
params_i = params_i.reshape([-1,9])
params_i = np.median(params_i,axis=0)
print(params_i)

# plot with the median of params_g
lmy = np.linspace(1.0,1.12)
lmx = np.sqrt(lmy)
lmz = 1.0/(lmx*lmy)

print(lmx)
print(lmy)

sgmx_pr, sgmy_pr = GOH_vmap2(params_i, lmx, lmy, lmz)

fig, ax = plt.subplots(1,2,figsize=(12,5))
ax[0].plot(lmy,sgmx_pr)
ax[1].plot(lmy,sgmy_pr)

for i_sample in [0,1,2]:   

    idx = data[0,:]==i_sample
    x = data[2,idx]
    # print(x)
    sgmx_gt = data[3,idx] ##sigma real

    sgmy_gt = data[4,idx]
    ax[1].plot(x,sgmy_gt,'.')
    ax[0].plot(x,sgmx_gt,'.')

fig.suptitle('TE data', fontsize= 20)
plt.savefig("TE_data_pred",dpi=300)